# Rna A-to-I editing using Rediknowntool from Katana etl. and overlap high #confidence targets from Gabay et al. 


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import stats
import os
import seaborn as sns
import math



In [2]:
# get files from folder
kanataFolder="./kanataRediKnown/"
os.chdir(kanataFolder)
onlyfiles = [f for f in os.listdir(kanataFolder) if os.path.isfile(os.path.join(kanataFolder, f))]

In [3]:
# read files in dataframe
df = pd.DataFrame()
for i in range(0,len(onlyfiles)):
    fname=onlyfiles[i].split(".")[0] # assign sample name
    df0=pd.read_csv(onlyfiles[i], sep="\t") # read file
    df0['sampleID']=fname # assign sample to table
    df=df.append(df0) # append table

In [4]:
# read and clean up metadata
os.chdir('./sourceData/')
kanataMeta=pd.read_csv("kanataMeta.csv") # read kanata metadata
kanataMet=pd.DataFrame({'SRR': [],'Treatment': []}) # add SRR and treatment ids
kanataMeta.columns=['in']
kanataMet['SRR']=kanataMeta['in'].str.extract('SRR(.{,7})')[0]
kanataMet['Treatment']=kanataMeta["in"].str.split(":",expand=True)[1].str.split(";",expand=True)[0] # assign Treatment

In [5]:
df['sampleID']=df['sampleID'].str.split("SRR",expand=True)[1] # remove SRR from sampleID in df

In [6]:
df=df.merge(kanataMet,left_on="sampleID",right_on="SRR") # merge AEI and metadata

In [7]:
df['Group']=1
df['Group'].loc[df['Treatment'].str.contains("MM")]="MM"
df['Group'].loc[df['Treatment'].str.contains("Con")]="Con" # assign inoculates
df['dpi']=1 # assign dates
df['dpi'].loc[df['Treatment'].str.contains("120")]=120
df['dpi'].loc[df['Treatment'].str.contains("180")]=180

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
# split base counts
dfb=df["BaseCount[A,C,G,T]"].str.split(",",expand=True) # expand column
dfb.columns=['A',"C","G","T"] # rename columns
dfb['A']=dfb['A'].str.replace('[',"") # replace brackets
dfb['T']=dfb['T'].str.replace(']',"")

In [9]:
df[['A',"C","G","T"]]=dfb # merge tables

In [10]:
df[['A',"C","G","T"]]= df[['A',"C","G","T"]].astype("int")

In [11]:
#  no rows with T>=3
# keep only G>=3
df_raw=df
df=df.loc[df["G"]>=3]

In [12]:
# calculate editing frequency
df['editFreq']=df['G']/df['A']
# sort by editing frequency >= 1%
df.loc[df['editFreq']>=0.01] # all values are > 1 %

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Region,Position,Reference,Strand,Coverage-q30,MeanQ,"BaseCount[A,C,G,T]",AllSubs,Frequency,sampleID,SRR,Treatment,Group,dpi,A,C,G,T,editFreq
0,chrX,41654252,A,1,21,39.43,"[1, 0, 20, 0]",AG,0.95,5084286,5084286,Con120_rep4,Con,120,1,0,20,0,20.000000
1,chrX,72445292,A,0,21,39.71,"[5, 0, 16, 0]",AG,0.76,5084286,5084286,Con120_rep4,Con,120,5,0,16,0,3.200000
4,chr11,46272643,A,0,74,39.32,"[10, 1, 63, 0]",AG,0.86,5084286,5084286,Con120_rep4,Con,120,10,1,63,0,6.300000
5,chr17,45662949,A,0,41,39.17,"[11, 0, 30, 0]",AG,0.73,5084286,5084286,Con120_rep4,Con,120,11,0,30,0,2.727273
6,chr14,12411582,A,0,56,39.27,"[39, 0, 17, 0]",AG,0.30,5084286,5084286,Con120_rep4,Con,120,39,0,17,0,0.435897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,chr11,46272643,A,0,41,39.20,"[5, 0, 36, 0]",AG,0.88,5084288,5084288,MM120_rep2,MM,120,5,0,36,0,7.200000
333,chr17,45662949,A,0,14,40.00,"[4, 0, 10, 0]",AG,0.71,5084288,5084288,MM120_rep2,MM,120,4,0,10,0,2.500000
335,chr3,80692286,A,0,14,39.71,"[7, 0, 7, 0]",AG,0.50,5084288,5084288,MM120_rep2,MM,120,7,0,7,0,1.000000
337,chr3,80706908,A,0,30,38.17,"[24, 0, 6, 0]",AG,0.20,5084288,5084288,MM120_rep2,MM,120,24,0,6,0,0.250000


In [13]:
# test if edited site occurs at least in 2/3 of replicates, easy: positions are all unique
occurence = []
wpii=[120,120,180,180]
treat=['Con','MM']*2
loopi=pd.DataFrame({"treat":treat,"wpi":wpii})



In [14]:
df.columns=['Region', 'Position', 'Reference', 'Strand', 'Coverage-q30', 'MeanQ',
       'BaseCount[A,C,G,T]', 'AllSubs', 'Frequency', 'sampleID', 'SRR',
       'Treatment', 'Group', 'wpi', 'A', 'C', 'G', 'T', 'editFreq']

In [15]:
df['wpi']=df['wpi'].astype("str") # change dtype of wpi to str

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
# extract sites that are edited at least in 2/4 replicates

testSites=pd.DataFrame([])
for i in range(0,len(loopi)):
    tS=pd.DataFrame([])
    tS[['Position','Counts']]=pd.DataFrame(df['Position'].loc[(df['wpi']==str(loopi['wpi'].iloc[i])) & (df['Group']==loopi['treat'].iloc[i])].value_counts()).reset_index()
    tS['wpi']=loopi['wpi'].iloc[i]
    tS['Treatment']=loopi['treat'].iloc[i]
    testSites=testSites.append(tS)

In [17]:
testSites['wpi']=testSites['wpi'].astype('str')

In [18]:
# select for editing events which pass threshold
### threshold: 2/4 replicates for each condition
testSitesThr=testSites.loc[testSites['Counts']>=2] 

In [19]:
# make combination of position+wpi
uniqueEdSitesThr=pd.Series(list(zip(testSitesThr.wpi, testSitesThr.Position)))
#uniqueEdSitesThr=uniqueEdSitesThr.astype('str') # convert tuple to str

In [20]:
# keep df_raw for statistical comparisons
df_raw=df_raw.merge(kanataMet,left_on="sampleID",right_on="SRR") # merge tables

In [21]:
df_raw['wpi']=df_raw['dpi']

In [22]:
df_raw['wpiPosition']=list(zip(df_raw.wpi, df_raw.Position)) # combine wpi+Position in df_raw
df_raw['wpiPosition']=df_raw['wpiPosition'].astype('str') # convert tuple to str

In [23]:
testSitesThr['wpiPosition']=testSitesThr['wpi']+'+'+testSitesThr['Position'].astype('str')


/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
df_raw['wpiPosition']=df_raw['wpi'].astype('str')+'+'+df_raw['Position'].astype('str')

In [25]:
uniqueTestSitesThr=testSitesThr['wpiPosition'].unique() # retrieve unique editing sites
uniqueTestSitesThr=list(uniqueTestSitesThr) # convert unique editing sites to list

In [26]:
dfThr=df_raw.loc[df_raw['wpiPosition'].isin(uniqueTestSitesThr)] # keep editing sites that pass threshold

In [27]:
# clean up dfThr for export
dfThrClean=dfThr[['A','G','Group','wpi','wpiPosition']]



In [28]:
# test whether at least 2 replicates of non-edited site exist
uniqueSites=dfThrClean['wpiPosition'].unique()
excludeSites=[]
for i in range(0,len(uniqueSites)):
    uS=dfThrClean.loc[dfThrClean['wpiPosition']==uniqueSites[i]]
    if len(uS['Group'].loc[uS['Group']=="MM"]) < 2:
        #print("In wpiPosition {} treated with NBH less than 2 replicates".format(uniqueSites[i]))
        excludeSites.append(uniqueSites[i])
    if len(uS['Group'].loc[uS['Group']=="Con"]) < 2:
        #print("In wpiPosition {} treated with RML6 less than 2 replicates".format(uniqueSites[i]))
         excludeSites.append(uniqueSites[i])

In [42]:
dfThrClean=dfThrClean.loc[~dfThrClean['wpiPosition'].isin(excludeSites)]
dfThrClean.to_csv('kanataRediEli.csv') # export dfThrClean -> now go to REDIT in R and perform testing

In [43]:
# analyse statistical testing with Redit, see R
dfP=pd.read_csv('./sourceData/kanataEditingSitesRedit.csv')

In [44]:
dfP[['wpi','site1']]=dfP['site'].str.split("+",expand=True)

In [45]:
from statsmodels.stats.multitest import multipletests # for fdr

In [46]:
wpiUnique=dfP['wpi'].unique()

In [47]:
# adjust for fdr < 0.05 
# preassign dataframe
tT=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]})
for i in range(0,len(wpiUnique)): # loop through all wpi
    tTtemp=pd.DataFrame({'wpi':[],'sig':[],'fdr':[],'pValue':[],'index':[]}) # preassign temp datafram
    # add sig and pVal to dataframe
    tTtemp['sig']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[0]
    tTtemp['fdr']=multipletests(dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]],alpha=0.05,method="fdr_bh")[1]
    tTtemp[['index','pValue']]=dfP['pValue'].loc[dfP['wpi']==wpiUnique[i]].reset_index()
    tTtemp['wpi']=wpiUnique[i]
    tT=tT.append(tTtemp)

In [48]:
kanataRediAllEvents=tT.merge(dfP,left_on="pValue",right_on="pValue") # merge sites and fdr on "pValue"

In [49]:
dictChr=df_raw[['Region','Position']] # add chromosome to aggregated data, make dictionary w position&chr
dictChr.drop_duplicates(subset="Position",inplace=True)

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
dictChr['Position']=dictChr['Position'].astype('str')
kanataExport=kanataRediAllEvents.merge(dictChr,left_on="site1",right_on="Position")

/Users/karlfrontzek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
eliConf=pd.read_excel('eliConfidenceSites.xlsx') # read gabay et al. high-confidence rna editing sites

In [52]:
# match annotation eli high confidence sites with kanataExport, chr positions are all unique
eliConf=eliConf[['chr_mm10','position_mm10','strand_mm10','gene_name_hg38']]

In [53]:
eliConf['position_mm10']=eliConf['position_mm10'].astype('str')

In [55]:
kanataExport.merge(eliConf,left_on="Position",right_on="position_mm10").to_csv('kanataExportFinal.csv')